In [1]:
from get_driver_for_crawling import *
from make_dir import *
import time
from bs4 import BeautifulSoup as BS

get_driver_for_crawling imported
make_dir imported


In [2]:
set_dir('data')
year = Select(SELECTOR(year_selector))
for curr_year in year_list:
    chgDir(base_dir)
    year.select_by_visible_text(curr_year)
    createFolder(curr_year)
    semester = Select(SELECTOR(semester_selector))
    chgFolder(curr_year)
    year_dir = os.getcwd()
    
    for curr_semester in semester_list:
        chgDir(year_dir)
        semester.select_by_visible_text(curr_semester)
        createFolder(curr_semester)
        chgFolder(curr_semester)
        semester_dir = os.getcwd()
        
        category1 = Select(SELECTOR(category1_selector))
        for op1 in category1.options:
            chgDir(semester_dir)
            category1_text = op1.text
            category1.select_by_visible_text(category1_text)
            createFolder(category1_text)
            category2 = Select(SELECTOR(category2_selector))
            chgFolder(category1_text)
            category1_dir = os.getcwd()
            for op2 in category2.options:
                chgDir(category1_dir)
                category2_text = op2.text
                if category2_text == "전체": continue
                category2.select_by_visible_text(category2_text)
                
                SEARCH_CLICK()
                time.sleep(0.5)
                
                pageinfo = SELECTOR(pageinfo_selector).text
                totalpos = int(pageinfo.split(' of ')[1])
                if totalpos == 0: continue
                    
                createFolder(category2_text)
                chgFolder(category2_text)
                
                totalpos -= 1
                curr_pos = 0
                for page in range(totalpos // 15 + 1):
                    for row_num in range(15):
                        SELECTOR(get_mileage_popup_selector(row_num)).click()
                        
                        switch_window(1)
                        html = driver.page_source
                        soup = BS(html, 'html.parser')
                        
                        val_list = []
                        for val in soup.select('td[class="BoxText_1_C"]'):
                            val_list.append(val.text)
                        
                        class_info_list = val_list[:18]
                        try:
                            class_code = class_info_list[0]
                        except:
                            driver.close()
                            switch_window(0)
                            curr_pos += 1
                            if curr_pos > totalpos: break
                            continue
                        
                        f = open(class_code + '_' + curr_year + '_' + curr_semester[0] + '.txt', 'w', -1, 'utf-8')
                        f.write('Class_Info\n')
                        f.write('\t'.join(class_info_title) + '\n')
                        f.write('\t'.join(class_info_list) + '\n')
                        
                        f.write('Mileage_Result\n')
                        f.write('\t'.join(mileage_rank_title) + '\n')
                        mileage_result_list = val_list[18:]
                        total_participant_num = len(mileage_result_list) // 11
                        for i in range(total_participant_num):
                            f.write('\t'.join(mileage_result_list[i*11:(i+1)*11]) + '\n')
                        
                        f.close()
                        driver.close()
                        switch_window(0)
                        
                        curr_pos += 1
                        if curr_pos > totalpos: break  
                    
                    if curr_pos > totalpos: break
                    driver.find_element_by_xpath(nextpage_selector).click()
                    
                

                

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=86.0.4240.183)
